<a href="https://colab.research.google.com/github/Shumin-li-mcit/CIS5300/blob/main/Final_autopart_CIS5300_OL_hw4_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Vector Space Models 

## Due Date: Jun 12th
## Total Points: 76 (+ 15 extra credit)
- **Overview**: In this assignment you will implement many of the things you learned in [Chapter 6 of the textbook](https://web.stanford.edu/~jurafsky/slp3/6.pdf). If you haven't read it yet, now would be a good time to do that.  We'll wait.  Done?  Great, let's move on. 
    
    We will provide a corpus of Shakespeare plays, which you will use to create a term-document matrix and a term-context matrix. You'll implement a selection of the weighting methods and similarity metrics defined in the textbook. Ultimately, your goal is to use the resulting vectors to measure how similar Shakespeare plays are to each other, and to find words that are used in a similar fashion. All (or almost all) of the code you write will be direct implementations of concepts and equations described in [Chapter 6, sections 6.3-6.7](https://web.stanford.edu/~jurafsky/slp3/6.pdf).

    *All difficulties are easy when they are known.*
- **Delieverables:** 
    - Your implementations for the functions in the skeleton code (this notebook)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings

- [Vector Semantics](https://web.stanford.edu/~jurafsky/slp3/6.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft)
- [From Frequency to Meaning: Vector Space Models of Semantics](https://www.jair.org/media/2934/live-2934-4846-jair.pdf). Peter D. Turney and Patrick Pantel. Journal of Artificial Intelligence Research 2010
- [Paraphrasing for Style](http://www.aclweb.org/anthology/C12-1177) Wei Xu, Alan Ritter, Bill Dolan, Ralph Grisman, and Colin Cherry. Coling 2012
- [Evaluation methods for unsupervised word embeddings](http://www.aclweb.org/anthology/D15-1036) Tobias Schnabel, Igor Labutov, David Mimno, Thorsten Joachims. EMNLP 2015
- [Community Evaluation and Exchange of Word Vectors at wordvectors.org.](http://www.aclweb.org/anthology/P14-5004) Manaal Faruqui and Chris Dyer. ACL demos 2014

## FAQs

- When finding the top 10 similar items for a given target element, should I count the target element?  
    *No, do not count the target element.*

- How can I represent a character as a vector for calculating similarity?  
    *One reasonable way would be to do it much in the same way as for plays. You would just need to write code to segment out each character as the given code did for each play.*

- What kind of analysis can I perform on the female and male Shakespearean characters for the report?  
    *There can be various ways to go about this. You can look at PCA projections of the vectors representing the characters in the plays to see if the men and women cluster in distinct areas that are demarcated. Or you can look at the similarity scores among women, among men and compare it to the similarity scores between men and men to see if it is significantly different. To account for outliers, you can use averages in this case. Feel free to play around with different vector representations and different similarity measures.*

- How can I improve the performance and efficiency of my code?  
    *Try to use vectorized code wherever possible instead of using loops. You can refer to this resource on [vectorized code](http://www.cs.cornell.edu/courses/cs1112/2015sp/Exams/exam2/vectorizedCode.pdf).*

- How many documents should I consider for reporting similarity scores in the writeup?  
    *You need not report similarity scores for every pair of documents. A subset of similarity scores should be sufficient. For instance, you can include the top 10 of one or two documents.*

- How can I compute the similarity scores on SimLex999 data set, and compute their correlation with human judgments using Kendall's Tau?  
    *You can use simlex data to get a ranking list with your model and calculate the number of concordant and discordant pairs. You can refer to this resource on [Kendall’s Tau](https://www.statisticshowto.datasciencecentral.com/kendalls-tau/).*

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [4]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [5]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [6]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [7]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 65847590 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW4', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 65847590

Make sure this correct or we will not be able to store your grade


In [8]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Shumin Li'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [9]:
import os
import csv
import subprocess
import re
import random
import numpy as np

In [10]:
!gdown 1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ # https://drive.google.com/file/d/1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ/view?usp=share_link
!gdown 1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3 # https://drive.google.com/file/d/1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3/view?usp=share_link
!gdown 1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP # https://drive.google.com/file/d/1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1usS5D3heEt6MB60KEXoebuU6QLpwiqhJ
To: /content/play_names.txt
100% 550/550 [00:00<00:00, 2.57MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RI7YQIA0WUcMUZRY9Frh-Vcet-js2Cd3
To: /content/vocab.txt
100% 184k/184k [00:00<00:00, 94.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1c3wLkuSuKkaFIHOQFBK82awMP5H33uBP
To: /content/will_play_text.csv
100% 10.3M/10.3M [00:00<00:00, 151MB/s]


In [11]:
######### Helper Functions - DO NOT CHANGE #########
def read_in_shakespeare():
	'''Reads in the Shakespeare dataset processesit into a list of tuples.
		 Also reads in the vocab and play name lists from files.

	Each tuple consists of
	tuple[0]: The name of the play
	tuple[1] A line from the play as a list of tokenized words.

	Returns:
		tuples: A list of tuples in the above format.
		document_names: A list of the plays present in the corpus.
		vocab: A list of all tokens in the vocabulary.
	'''

	tuples = []

	with open('will_play_text.csv') as f:
		csv_reader = csv.reader(f, delimiter=';')
		for row in csv_reader:
			play_name = row[1]
			line = row[5]
			line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
			line_tokens = [token.lower() for token in line_tokens]

			tuples.append((play_name, line_tokens))

	with open('vocab.txt') as f:
		vocab =  [line.strip() for line in f]

	with open('play_names.txt') as f:
		document_names =  [line.strip() for line in f]

	return tuples, document_names, vocab

def get_row_vector(matrix, row_id):
	return matrix[row_id, :]

def get_column_vector(matrix, col_id):
	return matrix[:, col_id]
######### Helper Functions - DO NOT CHANGE #########

In [12]:
tuples, document_names, vocab = read_in_shakespeare()
print('tuples: ', tuples[:10], '\n', 'doc_names: ', document_names[:10], '\n', 'vocab: ', vocab[:10])

tuples:  [('Henry IV', ['act', 'i']), ('Henry IV', ['scene', 'i', 'london', 'the', 'palace']), ('Henry IV', ['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others']), ('Henry IV', ['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care']), ('Henry IV', ['find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant']), ('Henry IV', ['and', 'breathe', 'short', 'winded', 'accents', 'of', 'new', 'broils']), ('Henry IV', ['to', 'be', 'commenced', 'in', 'strands', 'afar', 'remote']), ('Henry IV', ['no', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil']), ('Henry IV', ['shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'children', 's', 'blood']), ('Henry IV', ['nor', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields'])] 
 doc_names:  ['Henry IV', 'Alls well that ends well', 'Loves Labours Lost', 'Taming of the Shrew', 'Antony and Cleopatra', 'Coriolanus', 'Hamlet', 'A Mid

In [13]:
sentences = []
for t in tuples[:10]:
  print(t[1])
  sentences.append(t[1])
  print(sentences)


['act', 'i']
[['act', 'i']]
['scene', 'i', 'london', 'the', 'palace']
[['act', 'i'], ['scene', 'i', 'london', 'the', 'palace']]
['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others']
[['act', 'i'], ['scene', 'i', 'london', 'the', 'palace'], ['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others']]
['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care']
[['act', 'i'], ['scene', 'i', 'london', 'the', 'palace'], ['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 'earl', 'of', 'westmoreland', 'sir', 'walter', 'blunt', 'and', 'others'], ['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care']]
['find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant']
[['act', 'i'], ['scene', 'i', 'london', 'the', 'palace'], ['enter', 'king', 'henry', 'lord', 'john', 'of', 'lancaster', 'the', 

# Section 1: Term-Document Matrix [14 points]


## 1.1 Creating Term-Document Matrix [10 points]

You will write code to compile a term-document matrix for Shakespeare's plays, following the description in the textbook.

> In a *term-document matrix*, each row represents a word in the vocabulary and each column represents a document from some collection. The figure below shows a small selection from a term-document matrix showing the occurrence of four words in four plays by Shakespeare. Each cell in this matrix represents the number of times a particular word (defined by the row) occurs in a particular document (defined by the column). Thus *clown* appeared 117 times in *Twelfth Night

|             | As You Like It |  Twelfth Night  | Julias Caesar | Henry V |
| :---------: |:--------------:| :-------------: | :----------:  | :-----: |
| **battle**	| 1 | 1 | 8 | 15 |
| **soldier**	| 2 | 2 | 12 | 36 |
| **fool**		| 37 | 58 | 1 | 5 |
| **crown**		| 5 | 117 | 0 | 0 |

The dimensions of your term-document matrix will be the number of documents $D$ (in this case, the number of Shakespeare's plays that we give you in the corpus by the number of unique word types $\|V\|$ in that collection.   The columns represent the documents, and the rows represent the words, and each cell represents the frequency of that word in that document. 

- **Problem 1.1: Term Document Matrix** [10 points]

    In your code you will write a function to `create_term_document_matrix`.  This will let you be the hit of your next dinner party by being able to answer trivia questions like *how many words did Shakespeare use?*, which may give us a hint to the answer to *How many words did Shakespeare know?*  The table will also tell you how many words Shakespeare used only once.  Did you know that there's a technical term for that?  In corpus linguistics they are called [*hapax legomena*](https://en.wikipedia.org/wiki/Hapax_legomenon), but I prefer the term *singleton*, because I don't like snooty Greek or Latin terms. 


In [14]:
from collections import Counter

def create_term_document_matrix(line_tuples, document_names, vocab):
	'''Returns a numpy array containing the term document matrix for the input lines.

	Inputs:
		line_tuples: A list of tuples, containing the name of the document and 
		a tokenized line from that document.
		document_names: A list of the document names
		vocab: A list of the tokens in the vocabulary

	# NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

	Let m = len(vocab) and n = len(document_names).

	Returns:
		td_matrix: A mxn numpy array where the number of rows is the number of words
				and each column corresponds to a document. A_ij contains the
				frequency with which word i occurs in document j.
	'''

	vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
	docname_to_id = dict(zip(document_names, range(0, len(document_names))))
 
	m = len(vocab)
	n = len(document_names)

	# initialize a matrix with n/document columns, m/vocab rows
	matrix  = np.zeros([m,n])
	
	# collapse words in the same document 
	doc_words = {}
	for t in line_tuples:
			if t[0] not in doc_words:
				doc_words[t[0]] = t[1]
			doc_words[t[0]].extend(t[1])

	# count words in each document, store in a dictionary as {document_name: Counter object}
	counter = {}
	for doc in doc_words:
		counter[doc] = Counter(doc_words[doc])

	# build the matrix
	for docname, j in docname_to_id.items():
		for vocab, i in vocab_to_id.items():
			for doc in counter:
				if doc == docname:
					matrix[i][j] = counter[doc][vocab] 

	# YOUR CODE HERE
	return matrix

In [15]:
tuples, document_names, vocab = read_in_shakespeare()
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
term_doc_matrix.shape # should be (22602, 36)

(22602, 36)

In [16]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q11_td_matrix', answer = term_doc_matrix[:5000, :35]) # we only check partial data

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Use Term-Document Matrix to Compare Documents [4 points]

The term-document matrix will also let us do cool things like figure out which plays are most similar to each other, by comparing the column vectors.  We could even look for outliers to see if some plays are so dissimilar from the rest of the canon that [maybe they weren't authored by Shakespeare after all](https://en.wikipedia.org/wiki/Shakespeare_authorship_question).  

Let's begin by considering the column representing each play.  Each column is a $\|V\|$-dimensional vector.  Let's use some math to define the similarity of these vectors.   By far the most common similarity metric is the cosine of the angle between the vectors.  The cosine similarity metric is defined in Section 6.3 of the textbook.

> The cosine, like most measures for vector similarity used in NLP, is based on the dot product operator from linear algebra, also called the inner product:

> dot-product($\vec{v}, \vec{w}) = \vec{v} \cdot \vec{w} = \sum_{i=1}^{N}{v_iw_i} = v_1w_1 +v_2w_2 +...+v_Nw_N$

> The dot product acts as a similarity metric because it will tend to be high just when the two vectors have large values in the same dimensions. Alternatively, vectors that have zeros in different dimensions (orthogonal vectors) will have a dot product of 0, representing their strong dissimilarity. 

> This raw dot-product, however, has a problem as a similarity metric: it favors long vectors. The vector length is defined as

> $\|\vec{v}\| = \sqrt{\sum_{i=1}^{N}{v_i^2}}$

> The dot product is higher if a vector is longer, with higher values in each dimension. More frequent words have longer vectors, since they tend to co-occur with more words and have higher co-occurrence values with each of them. The raw dot product thus will be higher for frequent words. But this is a problem; we would like a similarity metric that tells us how similar two words are regardless of their frequency.

> The simplest way to modify the dot product to normalize for the vector length is to divide the dot product by the lengths of each of the two vectors. This normalized dot product turns out to be the same as the cosine of the angle between the two vectors, following from the definition of the dot product between two vectors $\vec{v}$ and $\vec{w}$ as:

> $\vec{v} \cdot \vec{w} = \|\vec{v}\|\|\vec{w}\| cos \Theta$

> $\frac{\vec{v} \cdot \vec{w}}{\|\vec{v}\|\|\vec{w}\|} =  cos \Theta$

> The cosine similarity metric between two vectors $\vec{v}$ and $\vec{w}$ thus can be computed

> $cosine(\vec{v}, \vec{w}) = \frac{\vec{v} \cdot \vec{w}}{\|\vec{v}\| \|\vec{w}\|} = \frac{\sum_{i=1}^{N}{v_iw_i}}{\sqrt{\sum_{i=1}^{N}{v_i^2}} \sqrt{\sum_{i=1}^{N}{w_i^2}}} $

The cosine value ranges from 1 for vectors pointing in the same direction, through 0 for vectors that are orthogonal, to -1 for vectors pointing in opposite directions. Since our term-document matrix contains raw frequency counts, it is non-negative, so the cosine for its vectors will range from 0 to 1.  1 means that the vectors are identical, 0 means that they are totally dissimilar.  


- **Problem 1.2: Comparing plays**: Please implement `compute_cosine_similarity`, and for each play in the corpus, score how similar each other play is to it.  Which plays are the closet to each other in vector space (ignoring self similarity)?  Which plays are the most distant from each other? [4 points]


In [17]:
def compute_cosine_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''
	 
	# YOUR CODE HERE
	norm_1 = np.sqrt(np.sum(vector1**2)) 
	norm_2 = np.sqrt(np.sum(vector2**2))
 
	return np.dot(vector1,vector2)/(norm_1*norm_2)

In [18]:
def compute_cosine_similarity(vector1, vector2):
  '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''
  n1=int(vector1.T.dot(vector1))
  n2=int(vector2.T.dot(vector2))
  if(n1==0 or n2==0):  
    sim = 0
  else:
    sim = float(vector1.T.dot(vector2))/(np.sqrt(n1)*np.sqrt(n2))
  return sim

In [19]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q12_cos_sim', answer = compute_cosine_similarity)

You earned 0/4 points.

But, don't worry, you can re-submit and we will keep only your latest score.


# Section 2: Measuring word similarity [10 points]
Next, we're going to see how we can represent words as vectors in vector space.  This will give us a way of representing some aspects of the *meaning* of words, by measuring the similarity of their vectors. 

In our term-document matrix, the rows are word vectors.  Instead of a $\|V\|$-dimensional vector, these row vectors only have $D$ dimensions.  Do you think that's enough to represent the meaning of words? *(Spoiler Alert: no!)*


## 2.1 Term-Context Matrix [10 points]
Instead of using a term-document matrix, a more common way of computing word similarity is by constructing a term-context matrix (also called a word-word matrix), where columns are labeled by words rather than documents.  The dimensionality of this kind of a matrix is $\|V\|$ by $\|V\|$.  Each cell represents how often the word in the row (the target word) co-occurs with the word in the column (the context) in a training corpus.  

- **Problem 2:** For this part of the assignment, you should write the `create_term_context_matrix` function.  This function specifies the size word window around the target word that you will use to gather its contexts.  For instance, if you set that variable to be 4, then you will use 4 words to the left of the target word, and 4 words to its right for the context.  In this case, the cell represents the number of times in Shakespeare's plays the column word occurs in +/-4 word window around the row word. [10 points]

    You can now re-compute the most similar words for your test words using the row vectors in your term-context matrix instead of your term-document matrix.  What is the dimensionality of your word vectors now?  Do the most similar words make more sense than before?

In [20]:
def create_term_context_matrix(line_tuples, vocab, context_window_size=1):
	'''Returns a numpy array containing the term context matrix for the input lines.

	Inputs:
		line_tuples: A list of tuples, containing the name of the document and 
		a tokenized line from that document.
		vocab: A list of the tokens in the vocabulary

	# NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

	Let n = len(vocab).

	Returns:
		tc_matrix: A nxn numpy array where A_ij contains the frequency with which
				word j was found within context_window_size to the left or right of
				word i in any sentence in the tuples.
	'''
	n = len(vocab)
	vocab_to_id = dict(zip(vocab, range(0, len(vocab))))

	# YOUR CODE HERE

	# initialize a matrix, ‖𝑉‖ by ‖𝑉‖
	matrix  = np.zeros([n,n])
	
	# collapse all words into one list, preserve words order
	sentences = []
	for t in line_tuples:
			sentences.append(t[1])

	# build the matrix
	for wrd_list in sentences:
		for focus_wrd_indx, focus_wrd in enumerate(wrd_list):
      # Get the indices of all the context words for the given focus word
			for contxt_wrd_indx in range((max(0,focus_wrd_indx - context_window_size)),(min(len(wrd_list),focus_wrd_indx + context_window_size +1))):
				if wrd_list[contxt_wrd_indx] in vocab_to_id and focus_wrd in vocab_to_id and contxt_wrd_indx != focus_wrd_indx:
					word_index = vocab_to_id[focus_wrd]
					c_index = vocab_to_id[wrd_list[contxt_wrd_indx]]
					matrix[word_index, c_index] += 1

	return matrix

In [21]:
tuples, document_names, vocab = read_in_shakespeare()
term_context_matrix = create_term_context_matrix(tuples, vocab, context_window_size = 2)
term_context_matrix.shape # should be (22602, 22602)

(22602, 22602)

In [22]:
# PennGrader - DO NOT CHANGE
test_cases = [
    [0, 0], # disliken, tribe
    [1317, 5124], # the, sword
    [15363, 10070], # love, you
    [4601, 15242], # good, night
    [8961, 6221], # hope, not
]

test_entries = [term_context_matrix[i, j] for i, j in test_cases]
print(test_entries)
grader.grade(test_case_id = 'test_q2_term_context_matrix', answer = test_entries)

[0.0, 49.0, 168.0, 117.0, 6.0]
Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 3: Weighting terms [28 points]
Your term-context matrix contains the raw frequency of the co-occurrence of two words in each cell.  Raw frequency turns out not to be the best way of measuring the association between words.  There are several methods for weighting words so that we get better results.  You should implement two weighting schemes:

* Positive pointwise mutual information (PPMI)
* Term frequency inverse document frequency (tf-idf)

These are defined in Section 6.2 of the textbook.

*Warning, calculating PPMI for your whole $\|V\|$-by-$\|V\|$ matrix might be slow. Our intrepid TA's implementation for PPMI takes about 10 minutes to compute all values. She always writes perfectly optimized code on her first try. You may improve performance by using matrix operations a la MATLAB.*

## 3.1 Weighting Schemes [20 points]

- **Problem 3.1.1:** Implementing `create_PPMI_matrix` function [10 points]

- **Problem 3.1.2:** Implementing `create_tf_idf_matrix` function [10 points]

In [23]:
def create_PPMI_matrix(term_context_matrix):
	'''Given a term context matrix, output a PPMI matrix.
	
	See section 6.2 in the textbook.
	
	Hint: Use numpy matrix and vector operations to speed up implementation.
				Please also add a small constant 1e-6 to your term_context_matrix to avoid having 0s
	
	Input:
		term_context_matrix: A nxn numpy array, where n is
				the numer of tokens in the vocab.
	
	Returns: A nxn numpy matrix, where A_ij is equal to the
		 point-wise mutual information between the ith word
		 and the jth word in the term_context_matrix.
	'''       
	
	# YOUR CODE HERE
	term_context_matrix += 1e-6
	joint_prob = term_context_matrix/np.sum(term_context_matrix)

	word_C = np.sum(term_context_matrix, axis = 1)
	word_prob = word_C / np.sum(word_C)

	context_C = np.sum(term_context_matrix, axis = 0)
	context_prob =context_C/np.sum(context_C)

	joint_exp = np.outer(word_prob, context_prob)
 
	PMI = np.log2(joint_prob/joint_exp)
	PMI[PMI < 0] = 0
	return PMI

def create_tf_idf_matrix(term_document_matrix):
	'''Given the term document matrix, output a tf-idf weighted version.

	See section 6.2 in the textbook.
	
	Hint: Use numpy matrix and vector operations to speed up implementation.

	Input:
		term_document_matrix: Numpy array where each column represents a document 
		and each row, the frequency of a word in that document.

	Returns:
		A numpy array with the same dimension as term_document_matrix, where
		A_ij is weighted by the inverse document frequency of document h.
	'''

	# YOUR CODE HERE
	N = term_document_matrix.shape[1]
	N = 1.0*N
	indicator = 1.0*(term_document_matrix > 0)
	df = np.sum(indicator,axis=1)
	idf = np.log10(N/df)
	matrix=np.multiply(term_document_matrix, np.transpose(np.array([idf,])))

	return matrix 

- Testing `create_tf_idf_matrix`

In [85]:
tuples, document_names, vocab = read_in_shakespeare()
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
tfidf_matrix = create_tf_idf_matrix(term_doc_matrix)
tfidf_matrix.shape # should be (22602, 36)

(22602, 36)

In [86]:
tfidf_matrix[:5, :5]

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.50514998, 0.        , 0.        , 1.20411998],
       [0.32585358, 0.32585358, 0.        , 0.        , 0.32585358],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [87]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q31_tfidf_matrix', answer = tfidf_matrix[:1001, :31]) # we only check part of the answer so that we won't blow up the RAM

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- Testing `create_PPMI_matrix`

In [88]:
tuples, document_names, vocab = read_in_shakespeare()
mini_vocab = ['dagger', 'run', 'the', 'bloody', 'sword'] # we only use part of the vocab so that you won't blow up the RAM
term_context_matrix = create_term_context_matrix(tuples, mini_vocab, context_window_size = 2)
PPMI_matrix = create_PPMI_matrix(term_context_matrix)
PPMI_matrix.shape # should be (5, 5)

(5, 5)

In [89]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q32_ppmi_matrix', answer = PPMI_matrix)

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 3.2 Other Similarity Measurements [8 points]
There are several ways of computing the similarity between two vectors.  In addition to writing a function to compute cosine similarity in Section 1, you should also write functions to `compute_jaccard_similarity` and `compute_dice_similarity`. You can check out the defintion of the [Jaccard measure here](https://en.wikipedia.org/wiki/Jaccard_index#Weighted_Jaccard_similarity_and_distance). And, dice similarity measure is given by (2 * J)/(J + 1) where J is Jaccard index. Please refer to this wikipedia link on [Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) for more details.


- **Problem 3.2.1:** Implementing `jaccard` similarity [4 points]

In [24]:
def compute_jaccard_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''
	
	# YOUR CODE HERE
	return np.sum(np.minimum(vector1,vector2))/(np.sum(np.maximum(vector1, vector2)))


In [25]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q33_jaccard_sim', answer = compute_jaccard_similarity)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Problem 3.2.2:** Implementing `dice` similarity [4 points]

In [26]:

def compute_dice_similarity(vector1, vector2):
	'''Computes the cosine similarity of the two input vectors.

	Inputs:
		vector1: A nx1 numpy array
		vector2: A nx1 numpy array

	Returns:
		A scalar similarity value.
	'''

	# YOUR CODE HERE
	return np.sum(2*np.minimum(vector1,vector2))/(np.sum(vector1) + np.sum(vector2))

In [93]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q34_dice_sim', answer = compute_dice_similarity)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 4: Ranking [10 points]
In this section, you will put everything together and **rank the playes and the words** based on their similarity to others.
- **Problem 4.1:** Implement `rank_plays` [5 points]

In [27]:
def rank_plays(target_play_index, term_document_matrix, similarity_fn):
	''' Ranks the similarity of all of the plays to the target play.

	Inputs:
		document_names: List of document names, corresponding to  
			term_document_matrix columns (i.e. name of document corresponding to 
			term_document_matrix[:,i] is given by document_names[i])
		target_play_index: The integer index of the play we want to compare all others against.
		term_document_matrix: The term-document matrix as a mxn numpy array.
		similarity_fn: Function that should be used to compared vectors for two
			documents. Either compute_dice_similarity, compute_jaccard_similarity, or
			compute_cosine_similarity.

	Returns:
		A length-n list of strings corresponding to play names,
		ordered by decreasing similarity to the play indexed by target_play_index
	'''
	# m: # of words; n: # of documents
	m, n = term_document_matrix.shape
	
	# YOUR CODE HERE
	target = term_document_matrix[:, target_play_index]
	sim_dic = dict((i, similarity_fn(target, term_document_matrix[:,i])) for i in range(n))
 
	sims_sort_dic = dict(sorted(sim_dic.items(), key=lambda item: item[1], reverse = True))
	sims_sort = np.array(list(sims_sort_dic.keys()))
	return sims_sort

In [95]:
# DO NOT CHANGE #
term_doc_matrix = create_term_document_matrix(tuples, document_names, vocab)
tfidf_matrix = create_tf_idf_matrix(term_doc_matrix)

In [96]:
# Test Case - DO NOT CHANGE #
target_play = 'Richard III'
play_index = document_names.index(target_play)
target_play_ranking = rank_plays(play_index, tfidf_matrix, compute_cosine_similarity)
target_play_ranking.shape # should be a list of 36 numbers, representing the sorted similarity compared to the target play, Richard III

(36,)

In [97]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q41_play_ranking', answer = target_play_ranking)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Problem 4.2:** Implement `rank_plays` [5 points]

In [28]:
def rank_words(target_word_index, matrix, similarity_fn):
	''' Ranks the similarity of all of the words to the target word.

	Inputs:
		vocab: List of terms, corresponding to target_word_index rows (i.e. word corresponding
			to target_word_index[i,:] is given by vocab[i])
		target_word_index: The index of the word we want to compare all others against.
		matrix: Numpy matrix where the ith row represents a vector embedding of the ith word.
		similarity_fn: Function that should be used to compared vectors for two word
			ebeddings. Either compute_dice_similarity, compute_jaccard_similarity, or
			compute_cosine_similarity.

	Returns:
		A length-n list of indices, ordered by decreasing similarity to the 
		target word indexed by word_index
	'''
	# YOUR CODE HERE

	# vocab number 
	n, n = matrix.shape

	target = matrix[:, target_word_index]
	sims_dic = dict((i, similarity_fn(target, matrix[:, i])) for i in range(n))
	sims_sort_dic = dict(sorted(sims_dic.items(), key=lambda item: item[1], reverse = True))
	print(list(sims_sort_dic.keys()))
	sims_sort = np.array(list(sims_sort_dic.keys()))
	return sims_sort

In [99]:
# Test Case - DO NOT CHANGE #
tuples, document_names, vocab = read_in_shakespeare()
N = 6000
tmp_vocab = vocab[:N] # we only use part of the vocab so that you won't blow up the RAM
term_context_matrix = create_term_context_matrix(tuples, tmp_vocab, context_window_size = 2)
tmp_PPMI_matrix = create_PPMI_matrix(term_context_matrix)

target_word = 'sword'
i = tmp_vocab.index(target_word)

target_word_ranking = rank_words(i, tmp_PPMI_matrix, compute_cosine_similarity)

[5124, 4119, 2421, 1388, 5403, 5024, 4000, 796, 4750, 3931, 746, 4310, 3325, 3873, 4802, 4600, 5500, 236, 2944, 2978, 1263, 2929, 464, 1228, 3991, 308, 3347, 971, 242, 1653, 4625, 3260, 3005, 1216, 5367, 2767, 4013, 758, 2828, 1854, 2899, 4570, 3827, 2827, 2653, 4172, 1010, 203, 1785, 1462, 117, 3254, 4201, 4644, 4209, 1961, 3001, 2269, 5481, 475, 3120, 1880, 843, 5634, 633, 2313, 5169, 1657, 3765, 4426, 2854, 77, 5771, 5554, 2006, 416, 1366, 400, 5300, 313, 3252, 5449, 228, 1469, 2694, 3964, 5757, 1735, 5427, 295, 388, 5132, 3567, 332, 2993, 1053, 2677, 4280, 4122, 1788, 2373, 5759, 97, 3436, 5060, 2172, 811, 4272, 908, 929, 1969, 3456, 2151, 56, 684, 629, 1386, 2783, 276, 4361, 5299, 397, 4598, 1933, 3165, 1585, 849, 2387, 1592, 156, 1715, 2223, 894, 4649, 5301, 5236, 5704, 269, 924, 4214, 3798, 3529, 1095, 3009, 2433, 2293, 1867, 939, 1781, 3791, 910, 4237, 5981, 4017, 4582, 2143, 2578, 3863, 3227, 5917, 4350, 2361, 1349, 2627, 3442, 364, 4037, 2248, 2712, 4309, 4577, 1835, 405, 148

In [100]:
target_word = 'sword'
i = tmp_vocab.index(target_word)

target_word_ranking = rank_words(i, tmp_PPMI_matrix, compute_cosine_similarity)
# target_word_ranking[:5]
np.delete(target_word_ranking, 3, 0)

[5124, 4119, 2421, 1388, 5403, 5024, 4000, 796, 4750, 3931, 746, 4310, 3325, 3873, 4802, 4600, 5500, 236, 2944, 2978, 1263, 2929, 464, 1228, 3991, 308, 3347, 971, 242, 1653, 4625, 3260, 3005, 1216, 5367, 2767, 4013, 758, 2828, 1854, 2899, 4570, 3827, 2827, 2653, 4172, 1010, 203, 1785, 1462, 117, 3254, 4201, 4644, 4209, 1961, 3001, 2269, 5481, 475, 3120, 1880, 843, 5634, 633, 2313, 5169, 1657, 3765, 4426, 2854, 77, 5771, 5554, 2006, 416, 1366, 400, 5300, 313, 3252, 5449, 228, 1469, 2694, 3964, 5757, 1735, 5427, 295, 388, 5132, 3567, 332, 2993, 1053, 2677, 4280, 4122, 1788, 2373, 5759, 97, 3436, 5060, 2172, 811, 4272, 908, 929, 1969, 3456, 2151, 56, 684, 629, 1386, 2783, 276, 4361, 5299, 397, 4598, 1933, 3165, 1585, 849, 2387, 1592, 156, 1715, 2223, 894, 4649, 5301, 5236, 5704, 269, 924, 4214, 3798, 3529, 1095, 3009, 2433, 2293, 1867, 939, 1781, 3791, 910, 4237, 5981, 4017, 4582, 2143, 2578, 3863, 3227, 5917, 4350, 2361, 1349, 2627, 3442, 364, 4037, 2248, 2712, 4309, 4577, 1835, 405, 148

array([5124, 4119, 2421, ..., 5990, 5996, 5999])

In [101]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q42_word_ranking', answer = target_word_ranking[:5])

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 5: Free-response Questions [10 points]

In the ranking tasks, play with different vector representations and different similarity functions. **Does one combination appear to work better than another? Do any interesting patterns emerge?**

Some patterns you could touch upon (you can pick 1 aspect to answer):
* The fourth column of `will_play_text.csv` contains the name of the character who spoke each line. Using the methods described above, **which characters are most similar? Least similar? **
* Shakespeare's plays are traditionally classified into [comedies, histories, and tragedies](https://en.wikipedia.org/wiki/Shakespeare%27s_plays). **Can you use these vector representations to cluster the plays?**
* Do the vector representations of **[female characters](https://en.wikipedia.org/wiki/Category:Female_Shakespearean_characters)** differ distinguishably from **[male ones](https://en.wikipedia.org/wiki/Category:Male_Shakespearean_characters)**?

**Answer question 1:**

We can reasonably assume similar characters would speak similar sentences/similar words. 

In [102]:
tuples, document_names, vocab = read_in_shakespeare()
vocab_to_id = dict(zip(vocab, range(0, len(vocab))))

def read_character_in_shakspeare():
    '''
    Construct a dictionary with character name as keys,
    (vocab_size, 1) array as values, each array is filled by the term frequency that 
    appears in the lines spoken by the character.

    Return: 
      char_term: the dictionary has the format described above.
    '''

    char_term={}
    with open('will_play_text.csv') as f:
      csv_reader = csv.reader(f, delimiter=';')
      for row in csv_reader:
        # collect character names as keys
        ch_name = row[4]

        # collect tokenized words spoken by the character 
        line = row[5]
        line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
        line_tokens = [token.lower() for token in line_tokens]

        # set value for each key as an vocab_size by 1 array
        if ch_name not in char_term:
          char_term[ch_name]=np.zeros((len(vocab), 1))
        
        # calculate tokens as values to fill the array
        for token in line_tokens: 
          char_term[ch_name][vocab_to_id[token]]+=1

      f.close()

    return char_term

In [103]:
def compute_similarity(c_term, sim_fn):
  '''
    Construct a dictionary with character pair as keys,
    their similarity socre as values.

    Input:
    c_term: the dictionary containing character name as keys,
    (vocab_size, 1) array as values, each array is filled by the term frequency that 
    appears in the lines spoken by the character.
    sim_fn: the similarity function to calculate character pair's similarity score.

    Return: 
      similairty: the dictionary has the format described above.
    '''

  # a list of all characters
  character = list(c_term.keys()) 
  num_char = len(c_term)

  # initialize the similarity dictionary
  similarity = {}
  
  # iterate each character and calculate their similarity with the rest of characters
  for i in range(num_char):
      for j in range(i+1, num_char):
        similarity[(character[i], character[j])] = sim_fn(c_term[character[i]], c_term[character[j]])
  return similarity


In [104]:
sim_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]
char_term = read_character_in_shakspeare()

for sim_fn in sim_fns:
    similarity_pair = compute_similarity(char_term, sim_fn)  

    rank = sorted(similarity_pair, key=similarity_pair.get, reverse=True) 

    print('with %s, most similar pair %s, least similar pair %s' % (sim_fn.__qualname__, rank[0], rank[-1]))

with compute_cosine_similarity, most similar pair ('Second Pirate', 'Outlaws'), least similar pair ('Outlaws', 'Mariner')
with compute_jaccard_similarity, most similar pair ('Second Pirate', 'Outlaws'), least similar pair ('Outlaws', 'Mariner')
with compute_dice_similarity, most similar pair ('Second Pirate', 'Outlaws'), least similar pair ('Outlaws', 'Mariner')


# Extra credit [15 points]

Quantifying the goodness of one vector space representation over another can be very difficult to do.  It might ultimately require testing how the different vector representations change the performance when used in a downstream task like question answering. A common way of quantifying the goodness of word vectors is to use them to compare the similarity of words with human similarity judgments, and then calculate the correlation of the two rankings.

If you would like extra credit on this assignment, you can quantify the goodness of each of the different vector space models that you produced (for instance by varying the size of the context window, picking PPMI or tf-idf, and selecting among cosine, Jaccard, and Dice).  You can calculate their scores on the [SimLex999 data set](https://www.cl.cam.ac.uk/~fh295/simlex.html), and compute their correlation with human judgments using [Kendall's Tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient).

- **Deliverables**:
    - In the following cells, explain what **experiments** you ran, and which **settings** *(i.e. size of the context window, picking PPMI or tf-idf, and selecting among cosine, Jaccard, and Dice)* had the highest correlation with human judgments. 

- **Hint**:
    - For SimLex999 dataset, you only need to use `word1`, `word2`, and `SimLex999` columns, where `SimLex999` column is a **similarity rating by human beings** (ranging from 0 to 10) between `word1` and `word2`.
    - For Kendal's Tau, you can use check [this python implementation](https://www.geeksforgeeks.org/python-kendall-rank-correlation-coefficient/#) for reference.



**Settings**:


1.   fixed context window: +/-2
2.   PPMI
3. try different similarity functions, including Cosine, Jaccard, and Dice

**Goal**:

Experiment which similarity function can generate a vector model that has the highest correlation with humann judgement



In [61]:
'''
  Settings:
    context window: +/-2; 
    PPMI; 
    Cosine, Jaccard, and Dice
  
  Experiment which similarity function can generate a vector model that has the highest correlation with humann judgement
  '''

def read_simlex999():
	'''Reads in the SimLex999 dataset and processes it into a list of tuples.

	Each tuple consists of
	tuple[0]: word1
	tuple[1]: word2
  tuple[2]: human similarity rating

	Returns:
		tuples: A list of tuples in the above format.
	'''

	tuples = []

	with open('SimLex-999.txt', 'r') as f:
		lines =  [line.strip() for line in f][1:]
		for line in lines:
			words = line.split('\t')
			word_1= words[0]
			word_2= words[1]
			rating = words[3]
			tuples.append((word_1, word_2, rating))
	return tuples

In [62]:
tuples_std = read_simlex999()
len(tuples_std) # expect 999

999

In [63]:
# clean up SimLex data
_, _, vocab = read_in_shakespeare()

std = {}
for tuple in tuples_std:
  if tuple[0] in vocab and tuple[1] in vocab:
    ''' for overlapping vocab of SimLex and 'wii_play_text.tsv',
    construct a dictionary with word pair as key, human rating as value
    '''
    std[(tuple[0], tuple[1])] = tuple[2]

# update vocab to the overlapping vocab between two dataset 
updated_vocab = set()
for tu in std.keys():
  updated_vocab.add(tu[0])
  updated_vocab.add(tu[1])
updated_vocab = list(updated_vocab)

In [64]:
len(updated_vocab)

724

In [65]:
# construct SimLex term-context matrix
n = len(updated_vocab)
vocab_to_id = dict(zip(updated_vocab, range(0, len(updated_vocab))))
std_matrix  = np.zeros([n,n])
for key, value in std.items():
  word, context = key
  word_index = vocab_to_id[word]
  c_index = vocab_to_id[context]
  std_matrix[word_index, c_index] = value

# adjust the similarity score when word and context are the same
for i in range(n):
  for j in range(n):
    if i == j:
      std_matrix[i][j] = 1.0

std_matrix.shape

(724, 724)

In [67]:
# PPMI matrix from 'wii_play_text.tsv'
tuples, _, _ = read_in_shakespeare()
term_context_matrix = create_term_context_matrix(tuples, updated_vocab, context_window_size = 2)
PPMI_matrix = create_PPMI_matrix(term_context_matrix)

PPMI_matrix.shape

(724, 724)

In [69]:
from scipy.stats import kendalltau

def compare_with_std(std_matrix, ppmi_matrix, sim_fn):
  '''
  Compare targeted dataset's term_term similarity with term_term human rated similarity based on SimLex dataset.
  
  Input:
    std_matrix: term_term similarity matrix made from SimLex dataset.
    ppmi_matrix: term_term PPMI weighted matrix made from the dataset to be compared with.
    sim_fn: similarity function used to calculate the similarity score for ppmi_matrix.
  
  Return:
    Kendall's Tau Correlation between standard and targeted vectors.
  '''
  ken = 0
  
  for i in range(n):
    # calculate similarity 
    target = ppmi_matrix[:, i]
    std = std_matrix[:, i]
    similarity = []
    for j in range(n):
      similarity.append(sim_fn(target, ppmi_matrix[:, j]))
    similarity = np.asarray(similarity)
    coff, _ = kendalltau(std, similarity)
    ken += coff
  
  ken = ken/n

  return ken


In [73]:
sim_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]
for sim_fn in sim_fns:
  ken = compare_with_std(std_matrix, PPMI_matrix, sim_fn)
  print('with %s, the correlation socre is %f' % (sim_fn.__qualname__, ken))

with compute_cosine_similarity, the correlation socre is 0.041452
with compute_jaccard_similarity, the correlation socre is 0.042262
with compute_dice_similarity, the correlation socre is 0.042262


**Conclusion**:

With a fixed context window = 2, using PPMI weighted matrix, Jaccard, and Dice genearte a slightly better word_word vector space comapred to Cosine for 'wii_play_text.tsv' dataset.

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework4.ipynb` and `homework4.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`